In [2]:
import pandas as pd
import re

In [89]:
# Load df from json
df = pd.read_json('./phones_all_data.json')

In [9]:
# Define a function to extract the brand
def extract_brand(name):
    if "apple" in name.lower():
        return "Apple"
    elif "samsung" in name.lower():
        return "Samsung"
    elif "motorola" in name.lower():
        return "Motorola"
    elif "nokia" in name.lower():
        return "Nokia"
    elif "kyocera" in name.lower():
        return "Kyocera"
    elif "google" in name.lower():
        return "Google"
    elif "tcl" in name.lower():
        return "TCL"
    elif "sonim" in name.lower():
        return "Sonim"

In [10]:
df['brand'] = df['name'].apply(extract_brand)

In [65]:
def convert_storage(storage_str):
    if storage_str == None:
        return None
    storage_str = storage_str.lower()
    if "," in storage_str:
        storage_list = [int(re.sub("[^0-9]", "", s).strip()) for s in storage_str.split(',')]
    elif "|" in storage_str:
        storage_list = [int(re.sub("[^0-9]", "", s).strip()) for s in storage_str.split('|')]
    else:
        storage_list = [int(re.sub("[^0-9]", "", storage_str).strip())]
    return storage_list

In [66]:
df['storage'] = df['storage'].apply(convert_storage)

In [69]:
df['used'] = df['name'].apply(lambda x: "owned" in x.lower())

In [74]:
# Remove (Certified Pre-Owned) from name
df['name'] = df['name'].apply(lambda x: re.sub(r"\(Certified Pre-Owned\)", "", x).strip())

In [84]:
def compute_screen_size(width, height):
    if width == None or height == None:
        return None
    if "Folded" not in width:
        width = float(width.strip("inches").strip("in.").strip("in").strip())
    else:
        width = float(width[width.find("Unfolded")::].strip("Unfolded:").strip("Unfolded").strip("in.").strip("in").strip())
    if "Folded" not in height:
        height = float(height.strip("inches").strip("in.").strip("in").strip())
    else:
        height = float(height[height.find("Unfolded")::].strip("Unfolded:").strip("Unfolded").strip("in.").strip("in").strip())
    return round((width**2 + height**2)**0.5, 2)
df['screen_size'] = df.apply(lambda x: compute_screen_size(x['width'], x['height']), axis=1)

Normalize camera categories

In [139]:
updated_camera_categories = {'general', 'modes', 'front', 'rear', "video"}

In [141]:
def normalize_camera_categories(camera_dict):
    if camera_dict == None:
        return None
    if 'Camera' in camera_dict.keys():
        camera_dict['general'] = camera_dict['Camera']
        del camera_dict['Camera']
    if 'Camera Modes' in camera_dict.keys():
        camera_dict['modes'] = camera_dict['Camera Modes']
        del camera_dict['Camera Modes']
    if 'Camera Mode' in camera_dict.keys():
        camera_dict['modes'] = camera_dict['Camera Mode']
        del camera_dict['Camera Mode']
    if "camera" in camera_dict.keys():
        camera_dict['general'] = camera_dict['camera']
        del camera_dict['camera']
    if "Main Lens" in camera_dict.keys():
        camera_dict['general'] = camera_dict['Main Lens']
        del camera_dict['Main Lens']
    if "Front Camera" in camera_dict.keys():
        camera_dict['front'] = camera_dict['Front Camera']
        del camera_dict['Front Camera']
    if "Rear Camera" in camera_dict.keys():
        camera_dict['rear'] = camera_dict['Rear Camera']
        del camera_dict['Rear Camera']
    if "Video" in camera_dict.keys():
        camera_dict['video'] = camera_dict['Video']
        del camera_dict['Video']
    for category in updated_camera_categories:
        if category not in camera_dict.keys():
            camera_dict[category] = None
    return camera_dict

In [142]:
df['camera'] = df['camera'].apply(normalize_camera_categories)

In [144]:
df.to_json('./phones_all_data.json.json', orient='records')

Compressing json for gpt input

In [4]:
df = pd.read_json('./phones_all_data.json')

# Remove depth, weight, height, and width columns
df = df.drop(columns=['depth', 'weight', 'height', 'width', 'operating_system'])

In [154]:
df.to_json('./phones_compressed_data.json', orient='records')